In [1]:
!pip install catboost==0.20.1

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import time
from catboost import CatBoostRegressor

DATA_PATH = "../input/ashrae-energy-prediction/"
LEAKDATA_PATH = "../input/leakdata/"



In [3]:
train = pd.read_pickle("train.pkl")
test = pd.read_pickle("test.pkl")

In [4]:
remove_features = ['year','timestamp','meter_reading', "precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed"]
features = []
for col in train.columns:
    if col not in remove_features:
        features.append(col)
#for col in ['building_id', 'site_id','meter','primary_use','square_feet','year_built','floor_count']:
#    try:features.remove(col)
#    except:None

In [5]:


categorical_feats = []

## Train

In [6]:
##building_id wise
import time
t1 = time.time()
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for building_id in train['building_id'].unique():
    a += 1
    print(a)
    for meter in train[train['building_id'] == building_id]['meter'].unique():
        train_small = train[(train['building_id'] == building_id) & (train['meter'] == meter)]
        test_small = test[(test['building_id'] == building_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

#             clf = lgb.LGBMRegressor(n_estimators=6000,
#                                         learning_rate=0.2,
#                                         feature_fraction=0.9,
#                                         subsample=0.2,  
#                                         subsample_freq=1,
#                                         num_leaves=20,
#                                         metric='rmse')
            clf = CatBoostRegressor(
                    iterations=5000, learning_rate=0.03,
                    depth=6, l2_leaf_reg=3,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=42,
                    task_type = "GPU",
                    devices='0:1')
            #if all(tr_y)!=0:
            if len(np.unique(tr_y))!=1:
                clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=100,verbose=2000,use_best_model=True)
                oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
                %time predictions += np.expm1(clf.predict(test_small[features]))
            else:
                v1 = np.unique(tr_y)[0]
                oof[val_idx] = np.array([v1]*len(val_idx))  
                predictions += np.expm1(np.array([v1]*len(test_small)))          
        test_small['meter_reading_building_id'] = predictions / num_folds
        test_small['meter_reading_building_id'] = test_small['meter_reading_building_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_building_id']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_building_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
oof2 = oof.copy()

t2 = time.time()
print('elasped time:',t2-t1)
#0.8102309160560927 lgb
#0.8999277997669956 cb

1
0:	learn: 0.3324466	test: 0.2626405	best: 0.2626405 (0)	total: 16.9ms	remaining: 1m 24s
bestTest = 0.2023004035
bestIteration = 34
Shrink model to first 35 iterations.
CPU times: user 26.5 ms, sys: 0 ns, total: 26.5 ms
Wall time: 6.15 ms
0:	learn: 0.3172230	test: 0.3037467	best: 0.3037467 (0)	total: 8.79ms	remaining: 43.9s
bestTest = 0.2149669129
bestIteration = 234
Shrink model to first 235 iterations.
CPU times: user 38.2 ms, sys: 0 ns, total: 38.2 ms
Wall time: 6.87 ms
0:	learn: 0.2828119	test: 0.3646640	best: 0.3646640 (0)	total: 8.88ms	remaining: 44.4s
bestTest = 0.2421801193
bestIteration = 246
Shrink model to first 247 iterations.
CPU times: user 19.2 ms, sys: 20.2 ms, total: 39.4 ms
Wall time: 6.87 ms
2
0:	learn: 0.9857831	test: 1.8695439	best: 1.8695439 (0)	total: 8.94ms	remaining: 44.7s
bestTest = 1.01100328
bestIteration = 565
Shrink model to first 566 iterations.
CPU times: user 44.9 ms, sys: 0 ns, total: 44.9 ms
Wall time: 7.52 ms
0:	learn: 1.2906467	test: 1.2753797	best

bestTest = 0.05138805308
bestIteration = 197
Shrink model to first 198 iterations.
CPU times: user 26.7 ms, sys: 7.47 ms, total: 34.2 ms
Wall time: 6.51 ms
0:	learn: 0.0965853	test: 0.1124859	best: 0.1124859 (0)	total: 8.89ms	remaining: 44.5s
bestTest = 0.06870810591
bestIteration = 173
Shrink model to first 174 iterations.
CPU times: user 24.1 ms, sys: 6.36 ms, total: 30.5 ms
Wall time: 6.55 ms
0:	learn: 0.9864607	test: 0.8906872	best: 0.8906872 (0)	total: 9.01ms	remaining: 45.1s
bestTest = 0.6575614811
bestIteration = 214
Shrink model to first 215 iterations.
CPU times: user 25.4 ms, sys: 6.76 ms, total: 32.2 ms
Wall time: 6.44 ms
0:	learn: 0.6528334	test: 1.3793987	best: 1.3793987 (0)	total: 8.86ms	remaining: 44.3s
bestTest = 1.26459849
bestIteration = 75
Shrink model to first 76 iterations.
CPU times: user 0 ns, sys: 29.9 ms, total: 29.9 ms
Wall time: 6.17 ms
0:	learn: 1.0106313	test: 0.7087473	best: 0.7087473 (0)	total: 8.9ms	remaining: 44.5s
bestTest = 0.7065267225
bestIteration 

bestTest = 0.2777326592
bestIteration = 1173
Shrink model to first 1174 iterations.
CPU times: user 56.2 ms, sys: 9.97 ms, total: 66.2 ms
Wall time: 9.58 ms
17
0:	learn: 0.5113102	test: 0.2070825	best: 0.2070825 (0)	total: 8.85ms	remaining: 44.2s
bestTest = 0.2047722605
bestIteration = 27
Shrink model to first 28 iterations.
CPU times: user 22.2 ms, sys: 5.85 ms, total: 28 ms
Wall time: 6.09 ms
0:	learn: 0.5125790	test: 0.2216824	best: 0.2216824 (0)	total: 8.95ms	remaining: 44.7s
2000:	learn: 0.0619875	test: 0.0678389	best: 0.0678389 (2000)	total: 12.8s	remaining: 19.1s
bestTest = 0.06667020618
bestIteration = 3437
Shrink model to first 3438 iterations.
CPU times: user 120 ms, sys: 6.38 ms, total: 126 ms
Wall time: 15.2 ms
0:	learn: 0.1859222	test: 0.7274224	best: 0.7274224 (0)	total: 8.8ms	remaining: 44s
bestTest = 0.6680046684
bestIteration = 400
Shrink model to first 401 iterations.
CPU times: user 32.1 ms, sys: 4.62 ms, total: 36.7 ms
Wall time: 7.33 ms
0:	learn: 1.8184405	test: 1.

0:	learn: 0.4530109	test: 0.7513354	best: 0.7513354 (0)	total: 8.87ms	remaining: 44.4s
bestTest = 0.5471880431
bestIteration = 220
Shrink model to first 221 iterations.
CPU times: user 27.5 ms, sys: 5.18 ms, total: 32.6 ms
Wall time: 6.46 ms
0:	learn: 0.4411877	test: 0.7549662	best: 0.7549662 (0)	total: 8.69ms	remaining: 43.4s
bestTest = 0.1924646832
bestIteration = 310
Shrink model to first 311 iterations.
CPU times: user 31.8 ms, sys: 5.68 ms, total: 37.5 ms
Wall time: 7 ms
0:	learn: 0.5216412	test: 0.4928478	best: 0.4928478 (0)	total: 8.73ms	remaining: 43.7s
bestTest = 0.347575385
bestIteration = 108
Shrink model to first 109 iterations.
CPU times: user 25.8 ms, sys: 4.74 ms, total: 30.6 ms
Wall time: 6.17 ms
28
0:	learn: 0.3110824	test: 0.3423452	best: 0.3423452 (0)	total: 8.9ms	remaining: 44.5s
bestTest = 0.1881371462
bestIteration = 630
Shrink model to first 631 iterations.
CPU times: user 49 ms, sys: 0 ns, total: 49 ms
Wall time: 7.9 ms
0:	learn: 0.3448155	test: 0.2631771	best: 

bestTest = 0.1864583415
bestIteration = 66
Shrink model to first 67 iterations.
CPU times: user 22.3 ms, sys: 6.69 ms, total: 29 ms
Wall time: 6.4 ms
0:	learn: 0.2419952	test: 0.2987371	best: 0.2987371 (0)	total: 8.89ms	remaining: 44.4s
bestTest = 0.2394309
bestIteration = 142
Shrink model to first 143 iterations.
CPU times: user 24.3 ms, sys: 4.1 ms, total: 28.4 ms
Wall time: 6.3 ms
37
0:	learn: 0.3178460	test: 0.2652933	best: 0.2652933 (0)	total: 9.02ms	remaining: 45.1s
bestTest = 0.2201419235
bestIteration = 42
Shrink model to first 43 iterations.
CPU times: user 20.8 ms, sys: 3.18 ms, total: 23.9 ms
Wall time: 5.84 ms
0:	learn: 0.2998881	test: 0.3034338	best: 0.3034338 (0)	total: 8.89ms	remaining: 44.5s
bestTest = 0.2162222213
bestIteration = 260
Shrink model to first 261 iterations.
CPU times: user 34.3 ms, sys: 0 ns, total: 34.3 ms
Wall time: 6.76 ms
0:	learn: 0.2828120	test: 0.3367888	best: 0.3367888 (0)	total: 8.95ms	remaining: 44.8s
bestTest = 0.2648473359
bestIteration = 65
S

...

bestTest = 0.1375378831
bestIteration = 223
Shrink model to first 224 iterations.
CPU times: user 32.9 ms, sys: 6.74 ms, total: 39.6 ms
Wall time: 8.56 ms
0:	learn: 0.2122048	test: 0.2660167	best: 0.2660167 (0)	total: 9.39ms	remaining: 46.9s
bestTest = 0.1297716828
bestIteration = 754
Shrink model to first 755 iterations.
CPU times: user 54.6 ms, sys: 0 ns, total: 54.6 ms
Wall time: 9.92 ms
1413
0:	learn: 0.1942568	test: 0.2111334	best: 0.2111334 (0)	total: 8.81ms	remaining: 44s
bestTest = 0.1057620936
bestIteration = 293
Shrink model to first 294 iterations.
CPU times: user 33.1 ms, sys: 4.64 ms, total: 37.8 ms
Wall time: 6.91 ms
0:	learn: 0.2029859	test: 0.1939705	best: 0.1939705 (0)	total: 8.66ms	remaining: 43.3s
bestTest = 0.05304663857
bestIteration = 257
Shrink model to first 258 iterations.
CPU times: user 40.3 ms, sys: 1.08 ms, total: 41.4 ms
Wall time: 8.77 ms
0:	learn: 0.2019075	test: 0.1951730	best: 0.1951730 (0)	total: 9.49ms	remaining: 47.4s
bestTest = 0.09988453117
bestIt

1424
0:	learn: 0.3685998	test: 0.3309747	best: 0.3309747 (0)	total: 8.9ms	remaining: 44.5s
bestTest = 0.1729549617
bestIteration = 103
Shrink model to first 104 iterations.
CPU times: user 22.3 ms, sys: 10.3 ms, total: 32.6 ms
Wall time: 6.42 ms
0:	learn: 0.3604332	test: 0.3459899	best: 0.3459899 (0)	total: 8.81ms	remaining: 44s
bestTest = 0.1871119372
bestIteration = 680
Shrink model to first 681 iterations.
CPU times: user 51.9 ms, sys: 0 ns, total: 51.9 ms
Wall time: 8 ms
0:	learn: 0.2819017	test: 0.5116805	best: 0.5116805 (0)	total: 8.77ms	remaining: 43.8s
bestTest = 0.3239894591
bestIteration = 250
Shrink model to first 251 iterations.
CPU times: user 27.7 ms, sys: 10.7 ms, total: 38.5 ms
Wall time: 6.85 ms
1425
0:	learn: 0.2452939	test: 0.2867104	best: 0.2867104 (0)	total: 8.92ms	remaining: 44.6s
bestTest = 0.1895447938
bestIteration = 332
Shrink model to first 333 iterations.
CPU times: user 31.6 ms, sys: 7.2 ms, total: 38.8 ms
Wall time: 7.25 ms
0:	learn: 0.2634025	test: 0.2622

0:	learn: 1.0028737	test: 1.3637316	best: 1.3637316 (0)	total: 8.67ms	remaining: 43.3s
bestTest = 1.285752849
bestIteration = 338
Shrink model to first 339 iterations.
CPU times: user 36.9 ms, sys: 0 ns, total: 36.9 ms
Wall time: 6.95 ms
0:	learn: 1.1525971	test: 1.2637471	best: 1.2637471 (0)	total: 8.85ms	remaining: 44.2s
bestTest = 0.651530697
bestIteration = 809
Shrink model to first 810 iterations.
CPU times: user 32.6 ms, sys: 11.7 ms, total: 44.3 ms
Wall time: 8.27 ms
1432
0:	learn: 0.3678129	test: 0.2362713	best: 0.2362713 (0)	total: 8.98ms	remaining: 44.9s
bestTest = 0.136100828
bestIteration = 49
Shrink model to first 50 iterations.
CPU times: user 28.8 ms, sys: 0 ns, total: 28.8 ms
Wall time: 6.07 ms
0:	learn: 0.3316425	test: 0.3427584	best: 0.3427584 (0)	total: 9.15ms	remaining: 45.8s
bestTest = 0.2214143228
bestIteration = 177
Shrink model to first 178 iterations.
CPU times: user 34.6 ms, sys: 0 ns, total: 34.6 ms
Wall time: 6.54 ms
0:	learn: 0.2082695	test: 0.5411613	best:

0:	learn: 0.1643187	test: 0.2831849	best: 0.2831849 (0)	total: 8.4ms	remaining: 42s
bestTest = 0.08131919808
bestIteration = 721
Shrink model to first 722 iterations.
CPU times: user 34.7 ms, sys: 10 ms, total: 44.7 ms
Wall time: 8.38 ms
1440
0:	learn: 0.1850385	test: 0.2051751	best: 0.2051751 (0)	total: 8.19ms	remaining: 40.9s
bestTest = 0.03430742536
bestIteration = 609
Shrink model to first 610 iterations.
CPU times: user 40 ms, sys: 8.21 ms, total: 48.2 ms
Wall time: 7.93 ms
0:	learn: 0.1930075	test: 0.1907199	best: 0.1907199 (0)	total: 8.47ms	remaining: 42.4s
bestTest = 0.06672782156
bestIteration = 1321
Shrink model to first 1322 iterations.
CPU times: user 66.2 ms, sys: 2.09 ms, total: 68.3 ms
Wall time: 10.7 ms
0:	learn: 0.1963352	test: 0.1818838	best: 0.1818838 (0)	total: 8.39ms	remaining: 42s
bestTest = 0.1108938212
bestIteration = 121
Shrink model to first 122 iterations.
CPU times: user 23 ms, sys: 4.72 ms, total: 27.7 ms
Wall time: 6.3 ms
1441
0:	learn: 0.3000114	test: 0.4

bestTest = 0.1283872851
bestIteration = 28
Shrink model to first 29 iterations.
CPU times: user 330 µs, sys: 26 ms, total: 26.3 ms
Wall time: 5.91 ms
0:	learn: 0.1607215	test: 0.2551108	best: 0.2551108 (0)	total: 8.96ms	remaining: 44.8s
bestTest = 0.2067622543
bestIteration = 608
Shrink model to first 609 iterations.
CPU times: user 42.8 ms, sys: 0 ns, total: 42.8 ms
Wall time: 7.67 ms
0:	learn: 0.1715849	test: 0.2386330	best: 0.2386330 (0)	total: 8.11ms	remaining: 40.5s
bestTest = 0.2043236982
bestIteration = 134
Shrink model to first 135 iterations.
CPU times: user 29.4 ms, sys: 0 ns, total: 29.4 ms
Wall time: 6.3 ms
1448
0:	learn: 0.4187120	test: 0.3533472	best: 0.3533472 (0)	total: 9.11ms	remaining: 45.5s
bestTest = 0.1869112003
bestIteration = 288
Shrink model to first 289 iterations.
CPU times: user 30.2 ms, sys: 6.21 ms, total: 36.4 ms
Wall time: 6.67 ms
0:	learn: 0.4125753	test: 0.3426672	best: 0.3426672 (0)	total: 8.98ms	remaining: 44.9s
bestTest = 0.2295808897
bestIteration =

In [7]:
##site wise
import time
t1 = time.time()
num_folds = 3
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for site_id in train['site_id'].unique():
    a += 1
    print(a)
    for meter in train[train['site_id'] == site_id]['meter'].unique():
        train_small = train[(train['site_id'] == site_id) & (train['meter'] == meter)]
        test_small = test[(test['site_id'] == site_id) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]

#             clf = get_clf(site_id)
            clf = CatBoostRegressor(
                    iterations=5000, learning_rate=0.03,
                    depth=6, l2_leaf_reg=3,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=42,
                    task_type = "GPU",
                    devices='0:1')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=100,verbose=2000,use_best_model=True)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_site_id'] = predictions / num_folds
        test_small['meter_reading_site_id'] = test_small['meter_reading_site_id'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_site_id']])
        tr_oof.append(train_small[['meter_reading_p']])
        

t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
# print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))

try:del test['meter_reading_site_id']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
oof1 = oof.copy()

t2 = time.time()
print('elasped time:',t2-t1)

#0.8864115837463974 lgb

#0.9193790471358488 cat

1
0:	learn: 1.1854424	test: 1.0564118	best: 1.0564118 (0)	total: 9.15ms	remaining: 45.8s
bestTest = 0.3267846653
bestIteration = 885
Shrink model to first 886 iterations.
CPU times: user 1.69 s, sys: 104 ms, total: 1.79 s
Wall time: 306 ms
0:	learn: 1.1154722	test: 1.2015538	best: 1.2015538 (0)	total: 4.42ms	remaining: 22.1s
bestTest = 0.4538200087
bestIteration = 991
Shrink model to first 992 iterations.
CPU times: user 1.86 s, sys: 117 ms, total: 1.98 s
Wall time: 322 ms
0:	learn: 1.1260182	test: 1.1803619	best: 1.1803619 (0)	total: 4.56ms	remaining: 22.8s
bestTest = 0.6575613919
bestIteration = 613
Shrink model to first 614 iterations.
CPU times: user 1.3 s, sys: 124 ms, total: 1.42 s
Wall time: 272 ms
0:	learn: 2.2546134	test: 2.5490180	best: 2.5490180 (0)	total: 3.34ms	remaining: 16.7s
bestTest = 1.483470986
bestIteration = 257
Shrink model to first 258 iterations.
CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 34.5 ms
0:	learn: 2.2633599	test: 2.4333832	best: 2.43338

bestTest = 1.59271697
bestIteration = 457
Shrink model to first 458 iterations.
CPU times: user 431 ms, sys: 55.4 ms, total: 486 ms
Wall time: 98.9 ms
0:	learn: 2.5431151	test: 3.2452779	best: 3.2452779 (0)	total: 3.47ms	remaining: 17.3s
bestTest = 2.13342722
bestIteration = 106
Shrink model to first 107 iterations.
CPU times: user 240 ms, sys: 17 ms, total: 257 ms
Wall time: 77.5 ms
0:	learn: 2.6405514	test: 2.8143396	best: 2.8143396 (0)	total: 3.45ms	remaining: 17.3s
bestTest = 2.124065609
bestIteration = 144
Shrink model to first 145 iterations.
CPU times: user 255 ms, sys: 37.9 ms, total: 292 ms
Wall time: 81.3 ms
7
0:	learn: 2.8219611	test: 3.5821716	best: 3.5821716 (0)	total: 3.34ms	remaining: 16.7s
bestTest = 1.165897898
bestIteration = 180
Shrink model to first 181 iterations.
CPU times: user 230 ms, sys: 12.1 ms, total: 242 ms
Wall time: 63.1 ms
0:	learn: 2.9024003	test: 3.5793129	best: 3.5793129 (0)	total: 3.4ms	remaining: 17s
bestTest = 1.584093381
bestIteration = 215
Shrink

0:	learn: 3.0535855	test: 3.1132021	best: 3.1132021 (0)	total: 3.35ms	remaining: 16.8s
bestTest = 2.787726361
bestIteration = 71
Shrink model to first 72 iterations.
CPU times: user 104 ms, sys: 4.85 ms, total: 109 ms
Wall time: 25.8 ms
11
0:	learn: 0.6608786	test: 0.6721098	best: 0.6721098 (0)	total: 13.7ms	remaining: 1m 8s
bestTest = 0.1312383821
bestIteration = 895
Shrink model to first 896 iterations.
CPU times: user 179 ms, sys: 8.6 ms, total: 187 ms
Wall time: 26.3 ms
0:	learn: 0.6503489	test: 0.6988122	best: 0.6988122 (0)	total: 13.2ms	remaining: 1m 5s
bestTest = 0.2328893618
bestIteration = 162
Shrink model to first 163 iterations.
CPU times: user 61.4 ms, sys: 10.5 ms, total: 71.9 ms
Wall time: 15.7 ms
0:	learn: 0.6617904	test: 0.6645180	best: 0.6645180 (0)	total: 12.7ms	remaining: 1m 3s
bestTest = 0.2584609016
bestIteration = 189
Shrink model to first 190 iterations.
CPU times: user 65.7 ms, sys: 16.4 ms, total: 82.1 ms
Wall time: 15.8 ms
0:	learn: 1.9847623	test: 2.4651063	b

0:	learn: 2.1996178	test: 2.3602606	best: 2.3602606 (0)	total: 3.98ms	remaining: 19.9s
bestTest = 1.119575048
bestIteration = 1134
Shrink model to first 1135 iterations.
CPU times: user 1.75 s, sys: 49 ms, total: 1.8 s
Wall time: 253 ms
0:	learn: 2.2380242	test: 2.2586755	best: 2.2586755 (0)	total: 4ms	remaining: 20s
bestTest = 1.072010296
bestIteration = 304
Shrink model to first 305 iterations.
CPU times: user 707 ms, sys: 67.6 ms, total: 775 ms
Wall time: 173 ms
15
0:	learn: 1.4986000	test: 1.7592613	best: 1.7592613 (0)	total: 10.8ms	remaining: 53.9s
bestTest = 1.16430598
bestIteration = 65
Shrink model to first 66 iterations.
CPU times: user 36.8 ms, sys: 5.4 ms, total: 42.2 ms
Wall time: 8.28 ms
0:	learn: 1.6974006	test: 1.3396652	best: 1.3396652 (0)	total: 10.1ms	remaining: 50.7s
bestTest = 0.7560969503
bestIteration = 653
Shrink model to first 654 iterations.
CPU times: user 68.8 ms, sys: 0 ns, total: 68.8 ms
Wall time: 11.9 ms
0:	learn: 1.4444825	test: 1.7975741	best: 1.7975741

In [8]:
##primary_use wise

import time
t1 = time.time()
num_folds = 4
#folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
folds = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
#train.dropna(subset=['meter_reading'], inplace=True)
t = []
tr_oof = []
a = 0
for primary_use in train['primary_use'].unique():
    a += 1
    print(a)
    for meter in train[train['primary_use'] == primary_use]['meter'].unique():
        #print('primary_use-meter:',primary_use,'-',meter)
        train_small = train[(train['primary_use'] == primary_use) & (train['meter'] == meter)]
        test_small = test[(test['primary_use'] == primary_use) & (test['meter'] == meter)]
        oof = np.zeros(len(train_small))
        predictions = np.zeros(len(test_small))
        target = np.log1p(train_small["meter_reading"])
        for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_small, train_small['month'])):
            #print('primary_use-meter-fold:',primary_use,'-',meter,'-',fold_)
            tr_x, tr_y = train_small.iloc[trn_idx][features], target.iloc[trn_idx]
            vl_x, vl_y = train_small.iloc[val_idx][features], target.iloc[val_idx]
            clf = CatBoostRegressor(
                    iterations=5000, learning_rate=0.03,
                    depth=6, l2_leaf_reg=3,
                    loss_function='RMSE',
                    eval_metric='RMSE',
                    random_seed=42,
                    task_type = "GPU",
                    devices='0:1')
            clf.fit(tr_x, tr_y,eval_set=[(vl_x, vl_y)],early_stopping_rounds=100,verbose=2000,use_best_model=True)
            oof[val_idx] = clf.predict(train_small.iloc[val_idx][features])
            %time predictions += np.expm1(clf.predict(test_small[features]))
        test_small['meter_reading_primary_use'] = predictions / num_folds
        test_small['meter_reading_primary_use'] = test_small['meter_reading_primary_use'].clip(train_small["meter_reading"].min(),train_small["meter_reading"].max())
        train_small['meter_reading_p'] = oof
        t.append(test_small[['meter_reading_primary_use']])
        tr_oof.append(train_small[['meter_reading_p']])
t = pd.concat(t)
t = t.sort_index()
oof = pd.concat(tr_oof)
oof = oof.sort_index()
#print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
try:del test['meter_reading_primary_use']
except:None
test = test.merge(t, left_index=True, right_index=True, how='left')
target = np.log1p(train["meter_reading"])
print(np.sqrt(mean_squared_error(target, oof['meter_reading_p'])))
t2 = time.time()
print('elasped time:',t2-t1)
oof3 = oof.copy()
#0.8636186523644124 lgb
#0.8692092058911276 cb

1
0:	learn: 1.5221169	test: 1.4830953	best: 1.4830953 (0)	total: 42ms	remaining: 3m 29s
bestTest = 0.3696141023
bestIteration = 1810
Shrink model to first 1811 iterations.
CPU times: user 28.2 s, sys: 1.06 s, total: 29.2 s
Wall time: 3.86 s
0:	learn: 1.5152341	test: 1.5048527	best: 1.5048527 (0)	total: 42.2ms	remaining: 3m 31s
2000:	learn: 0.2825312	test: 0.3165180	best: 0.3165180 (2000)	total: 50.5s	remaining: 1m 15s
bestTest = 0.3049606657
bestIteration = 3763
Shrink model to first 3764 iterations.
CPU times: user 55 s, sys: 999 ms, total: 56 s
Wall time: 5.54 s
0:	learn: 1.5043225	test: 1.5389745	best: 1.5389745 (0)	total: 42.4ms	remaining: 3m 31s
2000:	learn: 0.2856880	test: 0.3321263	best: 0.3321263 (2000)	total: 50.4s	remaining: 1m 15s
4000:	learn: 0.2503623	test: 0.3200017	best: 0.3198978 (3950)	total: 1m 40s	remaining: 25s
bestTest = 0.3198977571
bestIteration = 3950
Shrink model to first 3951 iterations.
CPU times: user 58 s, sys: 873 ms, total: 58.9 s
Wall time: 5.66 s
0:	lea

CPU times: user 796 ms, sys: 123 ms, total: 919 ms
Wall time: 192 ms
0:	learn: 2.4264625	test: 2.4993478	best: 2.4993478 (0)	total: 4.58ms	remaining: 22.9s
bestTest = 1.350486899
bestIteration = 557
Shrink model to first 558 iterations.
CPU times: user 1.09 s, sys: 67.8 ms, total: 1.16 s
Wall time: 215 ms
0:	learn: 2.4210604	test: 2.4845417	best: 2.4845417 (0)	total: 4.34ms	remaining: 21.7s
bestTest = 1.294184702
bestIteration = 193
Shrink model to first 194 iterations.
CPU times: user 620 ms, sys: 97 ms, total: 717 ms
Wall time: 202 ms
3
0:	learn: 1.6539426	test: 1.6618763	best: 1.6618763 (0)	total: 16.8ms	remaining: 1m 24s
2000:	learn: 0.3407441	test: 0.4448825	best: 0.4447540 (1984)	total: 19.3s	remaining: 28.9s
4000:	learn: 0.3061354	test: 0.4329649	best: 0.4329649 (4000)	total: 38.3s	remaining: 9.57s
4999:	learn: 0.2948121	test: 0.4304910	best: 0.4302814 (4929)	total: 47.9s	remaining: 0us
bestTest = 0.4302813719
bestIteration = 4929
Shrink model to first 4930 iterations.
CPU times

bestTest = 1.530094761
bestIteration = 790
Shrink model to first 791 iterations.
CPU times: user 662 ms, sys: 35.3 ms, total: 697 ms
Wall time: 114 ms
0:	learn: 2.5661369	test: 2.2509378	best: 2.2509378 (0)	total: 3.34ms	remaining: 16.7s
bestTest = 1.017850375
bestIteration = 156
Shrink model to first 157 iterations.
CPU times: user 123 ms, sys: 13.9 ms, total: 137 ms
Wall time: 27.9 ms
0:	learn: 2.4000163	test: 2.7043569	best: 2.7043569 (0)	total: 3.36ms	remaining: 16.8s
bestTest = 1.450106843
bestIteration = 633
Shrink model to first 634 iterations.
CPU times: user 283 ms, sys: 0 ns, total: 283 ms
Wall time: 41.1 ms
0:	learn: 2.3123320	test: 2.9618670	best: 2.9618670 (0)	total: 3.35ms	remaining: 16.7s
bestTest = 1.408970605
bestIteration = 495
Shrink model to first 496 iterations.
CPU times: user 221 ms, sys: 10.1 ms, total: 231 ms
Wall time: 34.5 ms
0:	learn: 2.5829957	test: 2.1396405	best: 2.1396405 (0)	total: 3.33ms	remaining: 16.7s
bestTest = 1.144507202
bestIteration = 406
Shrin

bestTest = 0.3344931826
bestIteration = 952
Shrink model to first 953 iterations.
CPU times: user 416 ms, sys: 17.2 ms, total: 433 ms
Wall time: 66.1 ms
0:	learn: 1.4616588	test: 1.6500480	best: 1.6500480 (0)	total: 3.35ms	remaining: 16.7s
bestTest = 0.2849830938
bestIteration = 1243
Shrink model to first 1244 iterations.
CPU times: user 523 ms, sys: 7.56 ms, total: 531 ms
Wall time: 79 ms
0:	learn: 1.4470375	test: 1.6911724	best: 1.6911724 (0)	total: 4.1ms	remaining: 20.5s
bestTest = 0.4327501933
bestIteration = 1026
Shrink model to first 1027 iterations.
CPU times: user 461 ms, sys: 4.18 ms, total: 465 ms
Wall time: 71.2 ms
0:	learn: 2.2497580	test: 1.7800858	best: 1.7800858 (0)	total: 11.3ms	remaining: 56.6s
bestTest = 1.654271598
bestIteration = 13
Shrink model to first 14 iterations.
CPU times: user 24.4 ms, sys: 10.2 ms, total: 34.7 ms
Wall time: 7.72 ms
0:	learn: 2.0730505	test: 2.3664080	best: 2.3664080 (0)	total: 11.9ms	remaining: 59.6s
bestTest = 1.7229321
bestIteration = 432

CPU times: user 1.72 s, sys: 50.8 ms, total: 1.77 s
Wall time: 206 ms
0:	learn: 0.8079062	test: 0.8257585	best: 0.8257585 (0)	total: 3.32ms	remaining: 16.6s
bestTest = 0.2360009498
bestIteration = 821
Shrink model to first 822 iterations.
CPU times: user 732 ms, sys: 60.5 ms, total: 792 ms
Wall time: 135 ms
0:	learn: 1.5745871	test: 1.7843395	best: 1.7843395 (0)	total: 11ms	remaining: 55s
bestTest = 0.5250912182
bestIteration = 359
Shrink model to first 360 iterations.
CPU times: user 57 ms, sys: 0 ns, total: 57 ms
Wall time: 10.6 ms
0:	learn: 1.6212466	test: 1.5578941	best: 1.5578941 (0)	total: 11ms	remaining: 54.9s
bestTest = 0.7037226213
bestIteration = 88
Shrink model to first 89 iterations.
CPU times: user 39 ms, sys: 869 µs, total: 39.9 ms
Wall time: 8.6 ms
0:	learn: 1.5543155	test: 1.8180777	best: 1.8180777 (0)	total: 11ms	remaining: 55s
bestTest = 0.7635993501
bestIteration = 133
Shrink model to first 134 iterations.
CPU times: user 34.8 ms, sys: 7.38 ms, total: 42.2 ms
Wall ti

bestTest = 0.3947695845
bestIteration = 728
Shrink model to first 729 iterations.
CPU times: user 51.9 ms, sys: 3.6 ms, total: 55.5 ms
Wall time: 8.44 ms
0:	learn: 0.7960580	test: 0.8925510	best: 0.8925510 (0)	total: 10.3ms	remaining: 51.7s
bestTest = 0.6858396593
bestIteration = 106
Shrink model to first 107 iterations.
CPU times: user 40.3 ms, sys: 1.15 ms, total: 41.5 ms
Wall time: 8.54 ms
0:	learn: 0.6833546	test: 1.1372578	best: 1.1372578 (0)	total: 9.89ms	remaining: 49.4s
bestTest = 0.814156656
bestIteration = 719
Shrink model to first 720 iterations.
CPU times: user 74.7 ms, sys: 1.02 ms, total: 75.7 ms
Wall time: 12.5 ms
0:	learn: 0.8121348	test: 0.8269133	best: 0.8269133 (0)	total: 9.79ms	remaining: 48.9s
bestTest = 0.3804782639
bestIteration = 86
Shrink model to first 87 iterations.
CPU times: user 28.9 ms, sys: 11.2 ms, total: 40.1 ms
Wall time: 8.37 ms
0:	learn: 0.8172835	test: 0.8240174	best: 0.8240174 (0)	total: 10ms	remaining: 50.1s
bestTest = 0.4860529699
bestIteration 

2000:	learn: 0.2160255	test: 0.4140976	best: 0.4140822 (1993)	total: 5.26s	remaining: 7.88s
bestTest = 0.4116578466
bestIteration = 2887
Shrink model to first 2888 iterations.
CPU times: user 903 ms, sys: 6.56 ms, total: 909 ms
Wall time: 84.4 ms
0:	learn: 1.1470827	test: 1.1228723	best: 1.1228723 (0)	total: 3.36ms	remaining: 16.8s
2000:	learn: 0.2215296	test: 0.3681556	best: 0.3680718 (1992)	total: 5.16s	remaining: 7.73s
bestTest = 0.3667169606
bestIteration = 2588
Shrink model to first 2589 iterations.
CPU times: user 811 ms, sys: 9.3 ms, total: 820 ms
Wall time: 82.1 ms
0:	learn: 1.1334955	test: 1.1686635	best: 1.1686635 (0)	total: 3.34ms	remaining: 16.7s
bestTest = 0.3833290885
bestIteration = 503
Shrink model to first 504 iterations.
CPU times: user 244 ms, sys: 0 ns, total: 244 ms
Wall time: 36.8 ms
0:	learn: 1.7988639	test: 1.5487815	best: 1.5487815 (0)	total: 11.3ms	remaining: 56.5s
bestTest = 0.5172977957
bestIteration = 800
Shrink model to first 801 iterations.
CPU times: use

In [9]:
# oof_save = oof1.copy()
# oof_save['meter_reading_site_id'] = oof1['meter_reading_p']
# oof_save['meter_reading_primary_use'] = oof3['meter_reading_p']
# oof_save['meter_reading_building_id'] = oof2['meter_reading_p']
# test_save = test[['meter_reading_site_id','meter_reading_primary_use','meter_reading_building_id']]
# test_save.to_csv("test_cb_save.csv")
# oof_save.to_csv("oof_cb_save.csv")


In [10]:
# print(np.sqrt(mean_squared_error(target, oof_save['meter_reading_site_id']*0.1+oof_save['meter_reading_building_id']*0.3+oof_save['meter_reading_primary_use']*0.6)))



In [11]:
# print(np.sqrt(mean_squared_error(target, oof1['meter_reading_p']*0.1+oof2['meter_reading_p']*0.3+oof3['meter_reading_p']*0.6)))



In [12]:
sub = pd.read_csv(DATA_PATH+"sample_submission.csv")
#sub["meter_reading"] = (test['meter_reading_site_id'] * 0.6 + test['meter_reading_primary_use'] * 0.2 + test['meter_reading_building_id'] * 0.2)
#sub.to_csv("best_more_deleting.gz",  index = False, compression= 'gzip')

In [13]:
test["meter_reading"] = test['meter_reading_site_id'] * 0.1 + test['meter_reading_primary_use'] * 0.6 + test['meter_reading_building_id'] * 0.3

In [14]:
###Postprocess 
test['meter_reading'] = [meter_r * 0.85 if site not in [0,1,2,3,4,15] else meter_r for meter_r,site in zip(test['meter_reading'], test['site_id'])]

In [15]:
leak_df = pd.read_csv(LEAKDATA_PATH+'leak.csv')
leak_df['timestamp'] = pd.to_datetime(leak_df["timestamp"])
leak_df.dropna(0, inplace=True)
leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]
leak_df['meter_reading_sc'] = leak_df['meter_reading'] 
del leak_df['meter_reading'] 
test = test.merge(leak_df, on=['building_id','meter','timestamp'], how='left')

In [16]:
test['meter_reading'] = [a if (a >= 0 and site != 4) or (a < 0 and site != 4) else b for a,b,site in zip(test['meter_reading_sc'], test['meter_reading'], test['site_id'])]

In [17]:
sub["meter_reading"] = test['meter_reading']
sub.to_csv("../sub/cb_3wise.csv",  index = False,float_format="%.4f")

In [18]:
sub.head()

,row_id,meter_reading
0,0,173.370293
1,1,53.512720
2,2,6.143042
3,3,101.701470
4,4,1141.240666
